# __JSC270 Lab 6 Part II: Intro to Scikit-Learn__

- __Note__: This part of the tutorial will be a bit less interactive. The goal here is to showcase some of the techniques you'll need, with an emphasis on fitting models.

## Contents

1. What is Scikit-learn?
2. Datasets
3. Some common preprocessing techniques
- Scaling
- One-hot Encoding
- Imputation
4. The Universality of SKLearn Design
5. Linear Regression (Fit, Fit-transform, Predict)
6. Logistic Regression


# __1. What is Scikit-Learn?__

<br>

- Scikit-learn is the largest and likely the most popular machine learning/ data analysis API (written for python)
-  It is written at a high level using packages we have or will cover in this class (numpy, scipy, matplotlib)
- It contains virtually all standard models and processes a data scientist will want to use (and many others)
- Because it is written at such a high level, most of the details are removed, allowing users to analyze data with very little code
- Like numpy and other main python packages, it has excellent documentation. I encourage you to visit the website:
[https://scikit-learn.org/stable/index.html]

- Though it contains libraries for almost every part of the data science pipeline (collection, cleaning, exploratory analysis, modelling, pipelining, deployment, etc.), today we'll mostly focus on fitting models

<br>

# __2. Datasets__

Today we're going to revisit the __California Housing Dataset__ which we saw a few weeks back.  The data contains a number of characteristics about the houses in California.  This dataset was derived from the 1990 U.S. census.

We'll run through the details of scikit learn with this dataset.

<br>


In [ ]:
# import our dataset
from sklearn.datasets import fetch_california_housing

# Remind yourself of the details of the dataset 
print(fetch_california_housing(as_frame = True).DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Let's start by loading in the dataset. Scikit learn actually has a number of built-in datasets that are great for practicing analysis skills.

Some things to note:
- Scikit-learn loads the data in as numpy arrays
- Unfortunately this means that you lose the metadata that would come with an external dataset

In [ ]:
import numpy as np
# Avoid displaying floats with scientific notation
np.set_printoptions(suppress=True)

# We load in the features and target separately
X,y = fetch_california_housing(return_X_y=True)

# Slight reshaping of target like before
n = X.shape[0] # number of observations
d = X.shape[1] # number of features
y = np.reshape(y, newshape=(n,1))

print('shape of features: ',X.shape)
print('shape of target: ', y.shape)
print(type(X))

print('This matrix contains data of type: ',X.dtype)

shape of features:  (20640, 8)
shape of target:  (20640, 1)
<class 'numpy.ndarray'>
This matrix contains data of type:  float64


Generally, your matrix of covariates should be $n \times p$, where n is the number of observations, and $p$ is the number of features. Thus each column is a different feature, and each row is an observation. Similarly your target vector should be $n \times 1$. This is what we have here (which is always a good sign). 
- To help with the missing metadeta, I've created a separate dictionary that contains the information we'll need to interpret features.
- All this information can be found in the [scikit-learn documentation](https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset) for this dataset.
- The order of keys corresponds exactly to the order of the columns in the feature matrix (with the exception of the 14th feature, which is actually the target: median house price)

In [ ]:
# Store variable info to help with interpretation later
variable_info = {0: 'median income in block group',
                 1: 'median house age in block group',
                 2: 'average number of rooms per household',
                 3: 'average number of bedrooms per household',
                 4: 'block group population',
                 5: 'average number of household members',
                 6: 'block group latitude',
                 7: 'block group longitude'}
print(variable_info[0])

median income in block group


Let's look at the first five observations of the first 5 features. Recall how to slice a numpy array:

In [ ]:
print('First 5 observations of the first 5 features:\n',X[:5,:5])
print('These are the first five variables:\n')
for i in range(5):
  print(i+1, variable_info[i])

First 5 observations of the first 5 features:
 [[   8.3252       41.            6.98412698    1.02380952  322.        ]
 [   8.3014       21.            6.23813708    0.97188049 2401.        ]
 [   7.2574       52.            8.28813559    1.07344633  496.        ]
 [   5.6431       52.            5.8173516     1.07305936  558.        ]
 [   3.8462       52.            6.28185328    1.08108108  565.        ]]
These are the first five variables:

1 median income in block group
2 median house age in block group
3 average number of rooms per household
4 average number of bedrooms per household
5 block group population


<br>

__Q: Are all these features actually numeric? If no, which ones are categorical?__ 

In [ ]:
# Add answer below 




<br>

Because this is a numpy array, all of our `numpy` functions from earlier still apply
- Let's get some summary statistics for each of our last 5 features:

In [ ]:
print('Maximum values:\n',np.max(X[:,-5:], axis = 0))
print('Minimum values:\n',np.min(X[:,-5:], axis=0))
print('Median values:\n',np.median(X[:,-5:], axis = 0))
print('Standard Deviations:\n', np.std(X[:,-5:], axis=0))

# Careful to specify the correct axis here

# Print the variables we're looking at:
for i in range(3, 8):
  print(i+1, variable_info[i])

Maximum values:
 [   34.06666667 35682.          1243.33333333    41.95
  -114.31      ]
Minimum values:
 [   0.33333333    3.            0.69230769   32.54       -124.35      ]
Median values:
 [   1.04878049 1166.            2.81811565   34.26       -118.49      ]
Standard Deviations:
 [   0.47389938 1132.43468776   10.38579796    2.13590065    2.00348319]
4 average number of bedrooms per household
5 block group population
6 average number of household members
7 block group latitude
8 block group longitude


<br>

These numbers seem reasonable.

__Warning: We should do a lot more preprocessing than you'll see in this notebook (and probably additional feature engineering too). Demonstrating the design of Sci-kit learn and showing you how to fit models is the focus here, but in reality exploring and cleaning the data will take up most of your time. This dataset is fairly clean, with no missing values, but I will touch on some cleaning techniques shortly.__

<br>

<br>

<br>

# __3. Some Common Preprocessing Techniques__
- Many of these techniques do not apply to our dataset, but I'll generate additional random covariates (that will not be used for modelling) to show you how they work.

<br>

## Categorical encoding

- Although the data we loaded in earlier has some categorical variables, they are all binary (0 or 1). If we did have a categorical variable with, say, 5 categories, there are several ways we might choose to represent them. 

In [ ]:
# Generate a random covariate
# Recall that n is the number of observations
cat_variable = np.random.choice(['A','B','C','D','E'], size = n)
cat_variable = np.reshape(cat_variable,(n,1))

# Show the first 10 observations
print(cat_variable[:10])

[['A']
 ['C']
 ['B']
 ['A']
 ['D']
 ['E']
 ['B']
 ['C']
 ['D']
 ['C']]


Keeping our categories stored as strings might not be very helpful when we want to train a model. Virtually every model we'll use requires the input to be a number (even if that number represents a category). We can use Sci-kit learn's __ordinal encoder__ to convert these letters to numbers.


In [ ]:
from sklearn.preprocessing import OrdinalEncoder

# Instantiate the ordinal encoder object using the OrdinalEncoder class
# You could provide specific encoding numbers as a hyperparameter if you want to
ord_encoder = OrdinalEncoder()

# fit the encoder to our data
cat_var_ord = ord_encoder.fit_transform(cat_variable)

# Show the first 10 observations
print(cat_var_ord[:10])

# What if I want to see how the old categories are represented here?
print('\nThese are the categories:\n',ord_encoder.categories_)


[[0.]
 [2.]
 [1.]
 [0.]
 [3.]
 [4.]
 [1.]
 [2.]
 [3.]
 [2.]]

These are the categories:
 [array(['A', 'B', 'C', 'D', 'E'], dtype='<U1')]


Representing a variable with ordinal encoding can be misleading if the order of the categories does not matter. Instead, we might want to make each category its own feature. This is called __one-hot encoding__, and under this representation, each of our five categories becomes a dummy variable:

In [ ]:
from sklearn.preprocessing import OneHotEncoder
# Again, instantiate the encoder object
ohot_encoder = OneHotEncoder()

# Fit the encoder to our data
cat_var_onehot = ohot_encoder.fit_transform(cat_variable)

# Print the new encoding
print('One-hot encoded:\n',cat_var_onehot[:10])

One-hot encoded:
   (0, 0)	1.0
  (1, 2)	1.0
  (2, 1)	1.0
  (3, 0)	1.0
  (4, 3)	1.0
  (5, 4)	1.0
  (6, 1)	1.0
  (7, 2)	1.0
  (8, 3)	1.0
  (9, 2)	1.0


The default output of `OneHotEncoder` is actually a scipy matrix, which prints a little differently (it's designed for very large, sparse datasets). To convert back to our usual array, we can use the `toarray()` method.

In [ ]:
print(cat_var_onehot.toarray()[:10])

[[1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]]


Notice that 1 feature has been turned into 5.

<br>

There are other encoders in the preproccessing library, but these two are the most common.

<br>

<br>

## Scaling and Standardizing

- Another common preprocessing step is standardizing features. Some models (e.g. neural nets, penalized regression, or distance-based algos like KNN) perform much better with standardized inputs.

- Standardization doesn't make much sense on categorical variables, so i'll generate 2 new features (which again will not be used for modelling) to demonstrate:

In [ ]:
# Generate the first feature
x1 = np.random.uniform(-50,0,n)
x2 = np.random.uniform(0,100,n)

# Reshape
x1, x2 = np.reshape(x1, (n,1)), np.reshape(x2, (n,1))

# Combine into a single feature matrix
X_sim = np.concatenate([x1,x2], axis=1)

# Print first 10 observations of both features
print('Simulated features:\n',X_sim[:10,:])

Simulated features:
 [[-15.7737627   69.57700997]
 [ -7.24649103  51.97785403]
 [-10.38765958  98.06595377]
 [-47.04775774  90.3108215 ]
 [ -6.91506328   5.90098095]
 [-35.13769311  86.12664076]
 [-28.54634805  25.58589018]
 [-18.88698621   2.18101792]
 [-24.56209018   9.86983259]
 [-20.38223114  74.17936896]]


We can see that the two features are very different. We can use SKLearn to scale them:

In [ ]:
from sklearn.preprocessing import StandardScaler

# Instantiate the transformer object
std_scaler = StandardScaler()

# Fit to data (subtract mean and divide by SD for each column)
scaled_X = std_scaler.fit_transform(X_sim)

# Print new features
print('New standardized features',scaled_X[:10,:])

New standardized features [[ 0.64044878  0.68656179]
 [ 1.23421448  0.07788876]
 [ 1.01549057  1.67186199]
 [-1.53720267  1.40364799]
 [ 1.25729225 -1.51569619]
 [-0.70788843  1.25893662]
 [-0.24892397 -0.83488664]
 [ 0.42367073 -1.64435242]
 [ 0.0285054  -1.37843205]
 [ 0.31955476  0.845736  ]]


We can see that the features have been standardized. There are other scalers (ie min-max scaling is common for discrete features), but we will not look at them here.

<br>


<br>

<br>

## Imputation
- One final preprocessing step is handling missing data. Since we have no missing data in the features above, let's generate some

In [ ]:
# Generate the domain of our feature, which can include a missing value
choices_of_x = list(range(51)) + [np.nan]

np.random.seed(10)
x_with_na = np.random.choice(choices_of_x, size=n)
x_with_na = np.reshape(x_with_na,(n,1))

# Print some values of our feature
print('Feature with missing values:\n',x_with_na[:20])

# How many missing values are there?
# Generate an identical vector containing TRUE(1) where missing, FALSE(0) where there is data
# Then just sum that vector to get the number of missing values
print('\nOut of {} observations, we have {} missing values'.format(n, np.sum(np.isnan(x_with_na)) ) )

Feature with missing values:
 [[ 9.]
 [36.]
 [15.]
 [ 0.]
 [49.]
 [28.]
 [25.]
 [29.]
 [48.]
 [29.]
 [49.]
 [ 8.]
 [ 9.]
 [ 0.]
 [42.]
 [40.]
 [36.]
 [nan]
 [16.]
 [36.]]

Out of 20640 observations, we have 399 missing values


We'v talked about missing data in lecture and lab.  One option to handle missing values is to impute them with reasonable values.

- For demonstration purposes, let's replace the `nan` values with the median of the column
- This is not the only choice, but it is a popular one 

In [ ]:
from sklearn.impute import SimpleImputer

# Instantiate the imputer transformer
median_imputer = SimpleImputer(strategy='median')

# Fit the imputation to the dataset
# Notice that since I already specified median earlier, I don't have to do so here
X_imputed = median_imputer.fit_transform(x_with_na)

# Print imputed data
print('Feature with imputed missing values:\n',X_imputed[:20])

# We can also access the median that our imputer used:
print('Medians used to replace missing values:\n',median_imputer.statistics_)

Feature with imputed missing values:
 [[ 9.]
 [36.]
 [15.]
 [ 0.]
 [49.]
 [28.]
 [25.]
 [29.]
 [48.]
 [29.]
 [49.]
 [ 8.]
 [ 9.]
 [ 0.]
 [42.]
 [40.]
 [36.]
 [25.]
 [16.]
 [36.]]
Medians used to replace missing values:
 [25.]


We can see that replacing missing values is very simple. Note that if we had more than one feature (p > 1), this imputer object would have computed the median for each feature, and replaced missing observations with the median belonging to the appropriate column.

<br>

<br>

## __QUESTIONS (About anything so far)?__

<br>

<br>

<br>

# __4. The Universality of Scikit-Learn Design__

- Many of these classes are very similar (You're probably noticing a general pattern)

- This is because the Scikit-Learn API is designed for consistency and simplicity

- Scikit-Learn has three general classes of objects:

    1. __Estimators__: Any object that estimates parameters based on data is an _estimator_ (for example, most models estimate at least one coefficient). Estimation is always done with the `.fit()` method, which always has only one required argument: the data. (For some supervised learning algorithms, the data may be passed as two arguments X,y). 

    2. __Transformers__: Some estimators also transform a dataset. All transformations are done with the `.transform()` method. Sometimes we can combine estimation and transformation using a `fit_transform()` method. The transformation generally relies on estimated/learned parameters (e.g our imputation estimated the medians before applying them). 

    3. __Predictors__: Given a dataset, some classes called _predictors_ can make predictions on new data. This is always done with the `predict()` method, which takes new data and returns predicted values. Every predictor also has a `score()` method, which evaluates the quality of predictions.

Some other useful information:
- Hyperparameters (ie things we choose as data analysts) are always available as variables, even after we instantiate the estimator (for example, our 'median' imputer strategy)
- Scikit-Learn almost always uses well-informed, sensible default values for optional arguments
- Datasets are always represented as NumPy arrays or SciPy sparse matrices, and hyperparameters are always regular python strings or numbers (no homemade classes or datatypes).

<br>

<br>

# __5. Linear Regression__
- You've all been waiting patiently so far
- Let's actually fit a model to our data
- Since our goal will be to predict the median housing price, let's start with a linear regression model to get a baseline
- We've already split our data into train and test sets

In [ ]:
from sklearn.linear_model import LinearRegression

# Instantiate the model object
linreg = LinearRegression(fit_intercept = True)

# Fit the model to our dataset
linreg.fit(X, y)

# What coefficients do we come up with?
print('Array of coefficients:\n',linreg.coef_)
print('Intercept:\n', linreg.intercept_)

# What is the R^2 value of the regression?
print('R-squared coefficient: ', linreg.score(X, y))

Array of coefficients:
 [[ 0.43669329  0.00943578 -0.10732204  0.64506569 -0.00000398 -0.00378654
  -0.42131438 -0.43451375]]
Intercept:
 [-36.94192021]
R-squared coefficient:  0.606232685199805


Some things to note here:
- The decision to include an intercept in the regression is fully automated, so I need only specify the hyperparameter once (no need for leading ones in the design matrix)
- All traditional elements of a regression fit are easily accessible
- All we've done here is fit a model
- To generate any kind of error or prediction, we'll use the `predict` method
- We'll also need some kind of loss/error function to evaluate performance

In [ ]:
from sklearn.metrics import mean_squared_error

# Generate predictions from training data
y_preds = linreg.predict(X)

# Generate Train and Test errors
mse = mean_squared_error(y, y_preds, squared = False)


# Setting squared=False gives Root MSE
print('Train RMSE: ', mse)

Train RMSE:  0.7241001216576387


- There are many other choices of evaluation metric beyond MSE (you can find many of them [here](https://scikit-learn.org/stable/modules/model_evaluation.html#)

<br>

<br>


<br>

<br>

# __6. Logistic Regression__
- Due to time constraints, I'm quickly going to pivot from regression (continuous response) to classification (in this case binary response)
- For purposes of illustration, we'll take our outcome variable and transform it to a binary variable to illustrate logistic regression with scikit learn
- Suppose we just want to know whether or not the house value is high. In this case, I define high as at or above the third quartile of all median housing values

In [ ]:
# Define new target threshold (third quartile)
c = np.quantile(y, q = 0.75)

# Generate new binary target
t = (y > c).astype(float)
print('Shape of new target', t.shape)
print('Number of high-priced neighbourhoods: ',np.sum(t))
print('First 5 labels:\n',t[:5])

Shape of new target (20640, 1)
Number of high-priced neighbourhoods:  5160.0
First 5 labels:
 [[1.]
 [1.]
 [1.]
 [1.]
 [1.]]


Pretty straightforward. Note that this has now become a classification task, a
we'll need a different model to account for the binary outcome.
- Let's try logistic classification
- We'll scale the features here for illustration

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Instantiate the transformer object
std_scaler = StandardScaler()

# Fit to data (subtract mean and divide by SD for each column)
X = std_scaler.fit_transform(X)

# Instantiate the model object
logit = LogisticRegression(fit_intercept = True, max_iter=1000, penalty = 'none')

# Fit the model to the data
logit.fit(X, t.ravel()) # Use reval so array is passed to the function

# Examine the coefficients and intercept
print('Logistic Regression Coefficients:\n',logit.coef_)
print('Intercept:\n', logit.intercept_)

# Get fitted values from training set
t_train_preds_lr = logit.predict(X)

# Print train and test errors
train_acc = accuracy_score(t, t_train_preds_lr)

print('Train Accuracy: ',train_acc)

Logistic Regression Coefficients:
 [[ 2.30114514  0.45812308 -0.65949543  0.84033213  0.09180904 -6.97973719
  -3.68618503 -3.40001348]]
Intercept:
 [-2.20978109]
Train Accuracy:  0.8734496124031008


We can see that the performance (at least based on accuracy) is not terrible.  We will spend an entire lecture on evaluating model performance more carefully including various metrics for binary classifiers, train/test splits, etc in a couple of weeks. 

Note the hyperparameters I chose here:
- I make sure to include the intercept
- I also specify the number of iterations for the numerical solver (almost all model classes use a numerical solver like SGD instead of a closed form solution)
- I used unpenalized regression (rather than ridge/lasso/etc which we haven't covered yet)
- Can we do better?

<br>

<br>



<br>

<br>

# Conclusion
- Today we looked at only a small demonstration of Scikit-Learn's coverage
- They have many more preprocessing functions, evaluation metrics and models
- They also allow easy stacking of each of these steps for when you want to deploy a full data science pipeline
- You should now have everything you need to complete A3 (but you still have lots of time)

<br>

<br>

# __QUESTIONS?__